In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [4]:
driver = webdriver.Chrome()
driver.get("https://mate.academy/en")

In [5]:
wait = WebDriverWait(driver, 5)
actions = ActionChains(driver)

courses = pd.DataFrame(columns=['name', 'link', 'description', 'format', 'num_modules', 'num_topics', 'duration'])

In [6]:
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "Button_secondary__pFIlL"))).click()

In [7]:
course_list = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'DropdownProfessionsItem_link__4NmVV')))
courses[['name', 'link']] = [[item.text, item.get_attribute("href")] for item in course_list if item.text != '']

In [8]:
def fill_table(i, attempts):
    for attempt in range(attempts):
        try:
            # redicrect to course page
            driver.get(courses['link'][i])
            
            # get description
            courses['description'][i] = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "typography_landingTextMain__Rc8BD"))).text
            
            # get available formates
            formats = [item.text for item in wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ButtonBody_buttonText__FMZEg")))]
            
            courses['format'] = courses['format'].fillna(value='')
            
            courses['format'][i] = ', '.join(filter(None, [
                "Full-time" if "I want to study full-time" in formats else None,
                "Flexible" if "I want to study flexibly" in formats else None
            ])) or 'No group openings'
            courses['format'][i] = courses['format'][i].lstrip(', ')
            
            # get number of modules
            button = driver.find_element(By.CLASS_NAME, "Button_neutral__ueX17")
            if button:
                actions.move_to_element(button).perform()
                button.send_keys(Keys.ENTER)
            modules = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "CourseModulesList_moduleListItem__HKJqw")))
            courses['num_modules'][i] = len(modules)
            
            # get number of topics
            topics = [item.text for item in wait.until(EC.presence_of_all_elements_located((By.XPATH, "//p[contains(@class, 'FactBlock_factDescription__Dy2k7') and contains(text(), 'Topics')]/preceding-sibling::p"))) if item.text != ''][0]
            courses['num_topics'][i] = topics
            
            # get duration of course   
            duration = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[contains(text(), 'month')]")))
            duration = [item.text.split()[0] for item in duration if item.text != '']
            courses['duration'][i] = f"{duration[0]}-{duration[1]} months" if 'Full-time' in courses['format'][i] else f"{duration[1]} months"
            return
        except Exception as e:
            print(f"Page: {courses['name'][i]};\nAttempt: {attempt};\nError: {e}")

<h5> Якщо не перезавантажувати сторінку, а спробувати закрити попап і знову зібрати всі дані зі сторінки, з'являються дивні помилки про те, що об'єкта на сторінці немає.

In [ ]:
for i in range(0, len(courses.index)):
    fill_table(i, 3)

In [ ]:
courses

,name,link,description,format,num_modules,num_topics,duration
0,UI/UX designer,https://mate.academy/en/courses/ui-ux,"Every time you use a website or app, you’re ex...","Full-time, Flexible",13,98,4-8 months
1,Data analyst,https://mate.academy/en/courses/data-analytics,Data Analysts use tools like SQL and Python to...,"Full-time, Flexible",16,126,3-5 months
2,QA engineer,https://mate.academy/en/courses/qa,"Every time you use a website or app, you benef...","Full-time, Flexible",23,176,3-5 months
3,Python developer,https://mate.academy/en/courses/python,"Every time you use a website or app, you benef...","Full-time, Flexible",21,193,4-7 months
4,Digital marketer,https://mate.academy/en/courses/digital-marketing,Every time you see an online ad or receive a t...,Flexible,11,73,4 months
5,Front-end developer,https://mate.academy/en/courses/frontend,"Every time you use a website or app, you’re in...","Full-time, Flexible",23,166,4-7 months
6,Full-stack developer,https://mate.academy/en/courses/fullstack,"Every time you use a website or app, you benef...","Full-time, Flexible",26,195,4-8 months
7,DevOps engineer,https://mate.academy/en/courses/devops,"Every time you use a website or app, you benef...",Flexible,21,183,4 months
8,Java developer,https://mate.academy/en/courses/java,"Every time you use a website or app, you benef...","Full-time, Flexible",19,192,4-8 months
9,Recruiter,https://mate.academy/en/courses/recruitment,Every time you find a new job or hire an emplo...,Flexible,9,48,4 months


In [ ]:
driver.quit()